In [1]:
#Code copied from: https://github.com/jaara/AI-blog/blob/master/CartPole-basic.py
# OpenGym CartPole-v0
# -------------------
#
# This code demonstrates use of a basic Q-network (without target network)
# to solve OpenGym CartPole-v0 problem.
#
# Made as part of blog series Let's make a DQN, available at: 
# https://jaromiru.com/2016/10/03/lets-make-a-dqn-implementation/
# 
# author: Jaromir Janisch, 2016


#--- enable this to run on GPU
# import os    
# os.environ['THEANO_FLAGS'] = "device=gpu,floatX=float32"  

import random, numpy, math, gym

#-------------------- BRAIN ---------------------------
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *


Using TensorFlow backend.
/Users/jmargoli/.pyenv/versions/3.5.2/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:

class Brain:
    def __init__(self, stateCnt, actionCnt):
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt

        self.model = self._createModel()
        # self.model.load_weights("cartpole-basic.h5")

    def _createModel(self):
        model = Sequential()

        model.add(Dense(units=64, activation='relu', input_shape=(stateCnt, )))
        model.add(Dense(output_dim=actionCnt, activation='linear'))

        opt = RMSprop(lr=0.00025)
        model.compile(loss='mse', optimizer=opt)

        return model

    def train(self, x, y, epoch=1, verbose=0):
        self.model.fit(x, y, batch_size=64, nb_epoch=epoch, verbose=verbose)

    def predict(self, s):
        return self.model.predict(s)

    def predictOne(self, s):
        return self.predict(s.reshape(1, self.stateCnt)).flatten()

#-------------------- MEMORY --------------------------
class Memory:   # stored as ( s, a, r, s_ )
    samples = []

    def __init__(self, capacity):
        self.capacity = capacity

    def add(self, sample):
        self.samples.append(sample)        

        if len(self.samples) > self.capacity:
            self.samples.pop(0)

    def sample(self, n):
        n = min(n, len(self.samples))
        return random.sample(self.samples, n)

#-------------------- AGENT ---------------------------
MEMORY_CAPACITY = 100000
BATCH_SIZE = 64

GAMMA = 0.99

MAX_EPSILON = 1
MIN_EPSILON = 0.01
LAMBDA = 0.001      # speed of decay

class Agent:
    steps = 0
    epsilon = MAX_EPSILON

    def __init__(self, stateCnt, actionCnt):
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt

        self.brain = Brain(stateCnt, actionCnt)
        self.memory = Memory(MEMORY_CAPACITY)
        
    def act(self, s):
        if random.random() < self.epsilon:
            return random.randint(0, self.actionCnt-1)
        else:
            return numpy.argmax(self.brain.predictOne(s))

    def observe(self, sample):  # in (s, a, r, s_) format
        self.memory.add(sample)        

        # slowly decrease Epsilon based on our eperience
        self.steps += 1
        self.epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * math.exp(-LAMBDA * self.steps)

    def replay(self):    
        batch = self.memory.sample(BATCH_SIZE)
        batchLen = len(batch)

        no_state = numpy.zeros(self.stateCnt)

        states = numpy.array([ o[0] for o in batch ])
        states_ = numpy.array([ (no_state if o[3] is None else o[3]) for o in batch ])

        p = self.brain.predict(states)
        p_ = self.brain.predict(states_)

        x = numpy.zeros((batchLen, self.stateCnt))
        y = numpy.zeros((batchLen, self.actionCnt))
        
        for i in range(batchLen):
            o = batch[i]
            s = o[0]; a = o[1]; r = o[2]; s_ = o[3]
            
            t = p[i]
            if s_ is None:
                t[a] = r
            else:
                t[a] = r + GAMMA * numpy.amax(p_[i])

            x[i] = s
            y[i] = t

        self.brain.train(x, y)

#-------------------- ENVIRONMENT ---------------------
class Environment:
    def __init__(self, problem):
        self.problem = problem
        self.env = gym.make(problem)

    def run(self, agent):
        s = self.env.reset()
        R = 0 

        while True:            
            self.env.render()

            a = agent.act(s)

            s_, r, done, info = self.env.step(a)

            if done: # terminal state
                s_ = None

            agent.observe( (s, a, r, s_) )
            agent.replay()            

            s = s_
            R += r

            if done:
                break

        print("Total reward:", R)

#-------------------- MAIN ----------------------------
PROBLEM = 'CartPole-v0'
env = Environment(PROBLEM)

stateCnt  = env.env.observation_space.shape[0]
actionCnt = env.env.action_space.n

agent = Agent(stateCnt, actionCnt)

try:
    while True:
        env.run(agent)
finally:
    agent.brain.model.save("cartpole-basic.h5")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Users/jmargoli/pythonEnvs/python3.5.2/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2, activation="linear")`
  
/Users/jmargoli/pythonEnvs/python3.5.2/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Total reward: 22.0
Total reward: 15.0
Total reward: 18.0
Total reward: 25.0
Total reward: 46.0
Total reward: 11.0
Total reward: 20.0
Total reward: 13.0
Total reward: 23.0
Total reward: 12.0
Total reward: 11.0
Total reward: 23.0
Total reward: 20.0
Total reward: 14.0
Total reward: 38.0
Total reward: 21.0
Total reward: 17.0
Total reward: 26.0
Total reward: 15.0
Total reward: 35.0
Total reward: 33.0
Total reward: 21.0
Total reward: 10.0
Total reward: 12.0
Total reward: 20.0
Total reward: 14.0
Total reward: 16.0
Total reward: 11.0
Total reward: 11.0
Total reward: 19.0
Total reward: 10.0
Total reward: 11.0
Total reward: 12.0
Total reward: 11.0
Total reward: 10.0
Total reward: 13.0
Total reward: 13.0
Total reward: 14.0
Total reward: 14.0
Total reward: 10.0
Total reward: 11.0
Total reward: 12.0
Total reward: 13.0
Total reward: 11.0
Total reward: 10.0
Total reward: 10.0
Total reward: 12.0
Total reward: 10.0
Total reward: 14.0
Total reward: 9.0
Total reward: 12.0
Total reward: 10.0
Total reward:

Total reward: 149.0
Total reward: 190.0
Total reward: 200.0
Total reward: 124.0
Total reward: 104.0
Total reward: 98.0
Total reward: 70.0
Total reward: 200.0
Total reward: 105.0
Total reward: 76.0
Total reward: 94.0
Total reward: 82.0
Total reward: 89.0
Total reward: 93.0
Total reward: 60.0
Total reward: 71.0
Total reward: 82.0
Total reward: 185.0
Total reward: 65.0
Total reward: 193.0
Total reward: 160.0
Total reward: 73.0
Total reward: 69.0
Total reward: 74.0
Total reward: 77.0
Total reward: 200.0
Total reward: 200.0
Total reward: 86.0
Total reward: 185.0
Total reward: 84.0
Total reward: 200.0
Total reward: 100.0
Total reward: 200.0
Total reward: 87.0
Total reward: 200.0
Total reward: 91.0
Total reward: 121.0
Total reward: 181.0
Total reward: 99.0
Total reward: 200.0
Total reward: 200.0
Total reward: 135.0
Total reward: 200.0
Total reward: 141.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 167.0
Total reward: 199.0


Total reward: 174.0
Total reward: 173.0
Total reward: 200.0
Total reward: 171.0
Total reward: 200.0
Total reward: 200.0
Total reward: 177.0
Total reward: 183.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 180.0
Total reward: 200.0
Total reward: 184.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 190.0
Total reward: 189.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 200.0
Total reward: 199.0
Total reward: 200.0
Total reward: 186.0
Total reward: 173.0
Total reward: 200.0
Total reward: 200.0
Total reward: 154.0
Total reward: 200.0
Total reward: 200.0


KeyboardInterrupt: 